In [12]:
import numpy as np 
import pandas as pd 
import json
import re

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [2]:
# Load the json file
                                                                                                                                                                                                            
with open("./data/00c2bfc7-57db-496e-9d5c-d62f8d8119e3.json") as f:
    data = json.load(f)

In [3]:
# Convert the json file to a pandas dataframe

df = pd.DataFrame(data)

df.head()

,id,text,title
0,7751000,M-137 was a state trunkline highway in the US ...,M-137 (Michigan highway)
1,7751001,"In sociology, dynamic density refers to the co...",Dynamic density
2,7751042,"Bert Robert Shepard (June 20, 1920 – June 16, ...",Bert Shepard
3,7751048,"Marc Fein (born Marc Alan Fein October 21, 196...",Marc Fein
4,7751062,Ghelamco Arena panorama indoor. The Ghelamco A...,Ghelamco Arena


In [4]:
# Clean the text data by removing special characters and converting to lower case 

def preprocess_data(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z0-9]', ' ', sentence)
    return sentence

cleaned_sentences = list(map(preprocess_data, list(df['text'].values)))

In [5]:
# Join the cleaned sentences to form a single string

joined_data = ' '.join(cleaned_sentences)

print(joined_data[:146])

m 137 was a state trunkline highway in the us state of michigan that served as a spur route to the interlochen center for the arts and interlochen


In [6]:
# Find the total number of unique words in the data

unique_words = list(set(joined_data.split()))

print(f"Total number of unique words : {len(unique_words)}")

Total number of unique words : 198868


In [18]:
# 198868 unique words is too much for us to handle, so we will only consider the top 10000 words for now

unique_words = unique_words[:100000]

In [19]:
unique_words[:10]

['zakharoff',
 'soutopenelo',
 'evas',
 'iuventutis',
 'maysfield',
 'nzie',
 'boschpoort',
 'indulgences',
 'catt',
 'craftsperson']

In [20]:
unique_words_dict = {word : i for i, word in enumerate(unique_words)}

In [21]:
tokenized_data = [unique_words_dict[word] for word in unique_words]

In [22]:
num_words = 7

input_data = []
output_data = []

for i in range(0, len(tokenized_data)-num_words):
    input_data.append(tokenized_data[i:i+num_words])
    output_data.append(tokenized_data[i+num_words])

In [23]:
model = Sequential()

model.add(Embedding(input_dim=len(unique_words), output_dim=100, input_length=num_words))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))

model.add(Dense(len(unique_words), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(np.array(input_data), np.array(output_data), epochs=10, batch_size=64)

Epoch 1/10
157/157 [==============================] - 19s 118ms/step - loss: 9.1756 - accuracy: 0.0000e+00
Epoch 2/10
157/157 [==============================] - 22s 141ms/step - loss: 9.2048 - accuracy: 0.0000e+00
Epoch 3/10
157/157 [==============================] - 22s 137ms/step - loss: 9.1764 - accuracy: 0.0000e+00
Epoch 4/10
157/157 [==============================] - 18s 117ms/step - loss: 9.0338 - accuracy: 0.0000e+00
Epoch 5/10
 39/157 [======>.......................] - ETA: 14s - loss: 8.7959 - accuracy: 0.0000e+00

KeyboardInterrupt: 